In [1]:
import json
import os
import sys
import numpy as np
import time
import urllib.request
import traceback
from transformers import LlamaForCausalLM, LlamaTokenizer
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
#from langchain.document_loaders import BSHTMLLoader
from src.overrides.html_bs import BSHTMLLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from bs4 import BeautifulSoup
import requests
import lxml
import logging
import inspect
import src.analyse as al
import src.queries as queries

import torch
torch.cuda.is_available()

True

In [2]:
no_factors= 5
no_links = 5
analysis_id = 'Crude_Oil'
subject = 'Crude Oil'
config_file = 'test'
model_props = {
        "name": "vicuna-33b.Q4_K_M.gguf", #llama-2-13b-chat.Q8_0.gguf",
        "n_gpu_layers": 52, #for vicuna, #43 for llama2
        "n_batch":1024,
        "n_ctx":4096
    }

direction = 'Increase'
factor_store = "FactorStore_0"
factors_analysis = "FactorsAnalysis_0"
effect_store = "EffectStore_0"
effect_analyses = [
            "EffectAnalysis_0"
       ]
func_dict = dict(inspect.getmembers(queries, inspect.isfunction))

In [3]:
func_dict

{'EffectAnalysis_0': <function src.queries.EffectAnalysis_0()>,
 'EffectAnalysis_1': <function src.queries.EffectAnalysis_1()>,
 'EffectAnalysis_2': <function src.queries.EffectAnalysis_2()>,
 'EffectStore_0': <function src.queries.EffectStore_0()>,
 'FactorStore_0': <function src.queries.FactorStore_0()>,
 'FactorsAnalysis_0': <function src.queries.FactorsAnalysis_0()>,
 'FactorsSplit_0': <function src.queries.FactorsSplit_0()>,
 'PriceStore_0': <function src.queries.PriceStore_0()>,
 'SentimentAnalysis_0': <function src.queries.SentimentAnalysis_0()>,
 'SentimentAnalysis_1': <function src.queries.SentimentAnalysis_1()>,
 'SentimentAnalysis_2': <function src.queries.SentimentAnalysis_2()>,
 'SentimentAnalysis_3': <function src.queries.SentimentAnalysis_3()>,
 'SentimentAnalysis_4': <function src.queries.SentimentAnalysis_4()>,
 'SentimentStore_0': <function src.queries.SentimentStore_0()>}

In [3]:
def ParseJSONResult(answer):
    try:
        res = json.loads(answer['result'])
        return res
    except Exception as e:
        logging.exception('failed to parse json: ')
        logging.exception(answer['result'])
        logging.exception(e)
        return None
        raise Exception(e)

def Parameterise(inp, subject = '', factor = '', no_factors = '', direction = ''):
    inp = inp.replace('{subject}', subject)
    inp = inp.replace('{factor}', factor)
    inp = inp.replace('{no_factors}', str(no_factors))
    inp = inp.replace('{direction}', direction)
    return inp
    
def CreateVectorDB(config_file, analysis_id, prefix):
    try:
        vector_db_path = os.path.join(os.getcwd(), 'output', config_file, 'html', analysis_id)
        vector_db_path = os.path.join(vector_db_path, prefix) + '\\'
        print('loading vector db at directory: ' + vector_db_path)
        loader = DirectoryLoader(vector_db_path, loader_cls=BSHTMLLoader, loader_kwargs={'open_encoding':'utf8','get_text_separator':'\n','get_text_strip':True})
        documents = loader.load()
        print('splitting documents')
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 250)
        docs = text_splitter.split_documents(documents)
        print('generating embeddings')
        embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
        vectorstore_faiss = FAISS.from_documents(docs, embeddings)
        return vectorstore_faiss
    except Exception as e:
        logging.exception("error generating embeddings")
        logging.exception(traceback.format_exc())
        raise Exception(e) 

def QueryBingAndCreateVectorStore(query, subject, factor, direction, no_links, config_file, analysis_id, prefix):
    bs = query()
    bs = (Parameterise(b, subject, factor, no_links, direction) for b in bs)
    #al.SearchBingGetLinksAndSave(bs, no_links, config_file, analysis_id, prefix)
    vs =  CreateVectorDB(config_file, analysis_id, prefix)
    return vs

def LoadModel(model_props, grammar = ''):
    model_path = os.path.join(os.getcwd(), 'models', model_props['name'])
    if grammar != '':
        grammar_path = os.path.join(os.getcwd(), 'models', grammar)
        llm = LlamaCpp(model_path=model_path, 
            temperature=0.0, 
            top_p=1,
            n_ctx=model_props['n_ctx'], 
            seed = 42,
            verbose=True, 
            n_gpu_layers=model_props['n_gpu_layers'],
            n_batch=model_props['n_batch'],
            grammar_path=grammar_path
        )
        return llm
    else:
        llm = LlamaCpp(model_path=model_path, 
            temperature=0.0, 
            top_p=1,
            n_ctx=model_props['n_ctx'], 
            seed = 42,
            verbose=True, 
            n_gpu_layers=model_props['n_gpu_layers'],
            n_batch=model_props['n_batch']
        )
        return llm

def LoadContextAndRunLLM(query, template, model_props, grammar, vectorstore_faiss, config_file):
    prompt = PromptTemplate(template = template, input_variables = ["context", "question"])
    model = LoadModel(model_props, grammar)
    qa = RetrievalQA.from_chain_type(
        llm=model,
        chain_type='stuff',
        retriever=vectorstore_faiss.as_retriever(search_type="similarity", search_kwargs={"k":10}),
        return_source_documents = True,
        chain_type_kwargs = {"prompt":prompt}
    )
    res = qa({"query": query})
    return res

def FindFactors(subject, query, direction, no_factors, no_links, vectorstore_faiss, model_props, config_file, analysis_id):
    q, t = query()
    q = Parameterise(q, subject, '', no_factors, direction) 
    answer = LoadContextAndRunLLM(q, t, model_props, 'json.gbnf', vectorstore_faiss, config_file)
    return answer
#llm = LoadModel(model_props, 'json.gbnf')

def LLMAnalyse(subject, factor, query, direction, no_links, vectorstore_faiss, model_props, config_file):
    q, t = query()
    q = Parameterise(q, subject, factor, '', direction)
    answer = LoadContextAndRunLLM(q, t, model_props, 'json.gbnf', vectorstore_faiss, config_file)
    return answer

In [5]:
## test llama cpp
llm = LoadModel(model_props)

prompt = """
Answer the question at the end using the following context. Answer in list form with each item just being a few words max.
<context>
speculation or changes in market sentiment
</context>
Q: Identify and seperate subjects in the context and create standalone bullet points. If there is only one subject, then there should only be one item in the list. A:
"""
res = llm(prompt)
res

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [4]:
def SentimentStore_0():
    return [
        "{subject} market news",
        "{subject} current market sentiment analysis",
        "recent reddit posts {subject}",
        "current trading analysis {subject}",
        "{subject} latest news and trends",
        "{subject} value outlook"
    ]
#prefix = direction + '_search'
# create vector store with bing search results for top n factors
vs = QueryBingAndCreateVectorStore(SentimentStore_0, subject,  '', '', no_links, config_file, analysis_id, 'sentiment_search')

loading vector db at directory: C:\Data\Code\llm_rag\output\test\html\Crude_Oil\sentiment_search\
splitting documents
generating embeddings


In [ ]:
query = 'Is the value of nymex crude oil likely to "Increase" or "Decrease" in the near future'
docs = vs.similarity_search(query)
docs

In [5]:
def SentimentAnalysis_0():
    p = 'Is the value of {subject} likely to "Increase" or "Decrease" in the near future'
    t = """
Use only the context below to give a one word answer to the question at the end. Answer in json form with the keys "answer" and "explanation".
<context>
{context}
</context>
Q: {question}? A:
"""
    return p, t
sentiment = LLMAnalyse(subject, '', SentimentAnalysis_0, '', no_links, vs, model_props, config_file)
# use llm query to extract the top n factors
#factors = FindFactors(subject, SentimentAnalysis_0, direction, no_factors, no_links, vs, model_props, config_file, analysis_id)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
from_string grammar:



root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number_28 
number_27 ::= [-

In [6]:
sentiment

{'query': 'Is the value of Crude Oil likely to "Increase" or "Decrease" in the near future',
 'result': '{\n"answer": "Decrease",\n"explanation": "The context mentions that increased U.S. production is expected to reach an all-time high in 2023, which will likely lead to a decline in oil prices."\n}',
 'source_documents': [Document(page_content='CONTINUE\nGo Back\nThe likely cause for declining oil prices is increased U.S. production, which is expected to reach an all-time high in 2023.\nSince the start of 2023 the price of U.S. crude oil has fallen by almost 10%, and now sits over 40% below its 2022 peak. Energy prices are dynamic and can fluctuate for a number of reasons. However, despite this unpredictability, many investors have been caught off guard by the weakness in crude, and are asking: what is the outlook for oil prices?', metadata={'source': 'C:\\Data\\Code\\llm_rag\\output\\test\\html\\Crude_Oil\\sentiment_search\\26.html', 'title': '\n        \n        \n        What is th

In [68]:
def ParseListResult(res):
    res = res.replace('"','')
    ls = res.split('\n')
    ls2 = []
    for it in ls:
        if it != '':
            it = it.lower()
            it = it.replace('1.','').replace('2.','').replace('3.','').replace('4.','').replace('5.','')
            it = it.replace('the first is ','').replace('the second is ','').replace('the third is ','').replace('the fourth is ','').replace('the fifth is ','')
            if it != '':
                ls2.append(it.strip())
    if len(ls2) == 1: #
        ls3 = ls2[0].split(',')
        ls2 = []
        for it in ls3:
            ls2.append(it.strip())

    for x in range(len(ls2)):
        ls2[x] = ls2[x].replace(',','')
    return ls2

ParseListResult('\n1. Increased demand for crude oil\n2. Decreased supply of crude oil\n3. Geopolitical issues affecting crude oil production\n4. Changes in currency exchange rates\n5. Speculation in the financial markets')

['increased demand for crude oil',
 'decreased supply of crude oil',
 'geopolitical issues affecting crude oil production',
 'changes in currency exchange rates',
 'speculation in the financial markets']

In [41]:
factors

{'query': 'Precisely what are the 5 biggest factors that cause the value of Nymex Crude Oil to Increase',
 'result': '\n1. The first is the geopolitical factor (geopolitics), including the supply and demand for crude oil, the second is the weather, the third is the storage, the fourth is the number of days it takes 5 days.',
 'source_documents': [Document(page_content='The nominal price of crude oil is just one factor involved in understanding the crude oil market.\xa0\n\n\n\nAccording to CME Group, which runs the NYMEX commodities market, the WTI/Brent Spread is influenced by four key factors:\n\n\n\nU.S. crude oil production levelsCrude oil supply-and-demand balance in the U.S.North Sea crude oil operationsGeopolitical issues in the international crude oil market\n\n\n   How World Events Can Affect Crude Oil Prices  \n\nPolitical shifts, weather events, and global health crises have been some of the biggest shock factors in the oil market.\n\n\n\n\nNote\n\nBecause of the coronavirus 

In [11]:
#ethereum
factors

{'query': 'Precisely what are the 5 biggest factors that cause the value of ethereum to Increase',
 'result': '[\n"Adoption and Network Usage",\n"Market Sentiment",\n"Competition",\n"Supply and Demand",\n"Government and Regulatory Developments"\n]',
 'source_documents': [Document(page_content='3 reasons why Ethereum has been rising faster than Bitcoin price in 2021', metadata={'source': 'C:\\Data\\Code\\llm_rag\\output\\test\\html\\ethereum\\Increase_search\\4.html', 'title': '3 reasons why Ethereum has been rising faster than Bitcoin price in 2021'}),
  Document(page_content='trading. In this article, we will discuss some of the major factors that can impact Ethereum’s price movements. \xa0    Related Articles         Spillovers in Volatility Among Cryptocurrency Time Series  November 18, 2023         RIF: Bridging Decentralized Services with Bitcoin  November 18, 2023         Adoption and Network Usage \xa0 One of the key factors that can influence Ethereum’s price is the adoption an

In [20]:
factors

{'query': 'what are the 5 biggest factors that cause the price of ethereum to increase',
 'result': '[\n]',
 'source_documents': [Document(page_content='3 reasons why Ethereum has been rising faster than Bitcoin price in 2021', metadata={'source': 'C:\\Data\\Code\\llm_rag\\output\\test\\html\\ethereum\\increase_search\\4.html', 'title': '3 reasons why Ethereum has been rising faster than Bitcoin price in 2021'}),
  Document(page_content='trading. In this article, we will discuss some of the major factors that can impact Ethereum’s price movements. \xa0    Related Articles         Spillovers in Volatility Among Cryptocurrency Time Series  November 18, 2023         RIF: Bridging Decentralized Services with Bitcoin  November 18, 2023         Adoption and Network Usage \xa0 One of the key factors that can influence Ethereum’s price is the adoption and usage of the network. The more people use Ethereum, the more valuable it becomes. As the demand for the network grows, the value of the unde

In [6]:
str(query.__name__)

'Effect_Q0'

In [41]:
def BingToActualURL(url):
    headers = {
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582" 
        }
    response = requests.get(url, headers = headers).text
    start_ind = response.index('var u = "')
    if start_ind >= 0:
        response = response[start_ind+9:]
        end_ind = response.find('";')
        if end_ind >=0:
            response = response[:end_ind]
            return response
    return None

def SearchBingGetLinksAndSave(queries, no_links, config_file, analysis_id, doc_prefix):
    try:
        doc_id = 0
        for query in queries:
            logging.info('running bing search for query: ' + query)
            url = "https://www.bing.com/search?form=QBRE&q="+query.replace(' ', '+')
            response = al.RequestURLAndSave(url, False, config_file, analysis_id, doc_prefix, doc_id)
            soup = BeautifulSoup(response, 'lxml')
            links = []
            for container in soup.select('.b_algo h2 a'):
                actURL = BingToActualURL(container['href']) # need to find the actual url
                if actURL is not None:
                    links.append(actURL)
            links = links[:no_links]

            return links
    except Exception as e:
        logging.exception("error running bing search and save results")
        logging.exception(traceback.format_exc())
        raise Exception(e) 

l = SearchBingGetLinksAndSave(['top 3 factors that cause the price of nymex crude oil to increase'], 3, 'prod',analysis_id,'incFact')

In [42]:
l

['https://www.forbes.com/sites/forbesbooksauthors/2021/01/25/factors-that-influence-pricing-of-oil-and-gas/#:~:text=Weather%2C%20political%20disturbances%2C%20supply%20problems%20%E2%80%94%20these%20factors,supply%20and%20demand%20levels%2C%20prices%20will%20level%20out.',
 'https://www.cmegroup.com/openmarkets/energy/2022/Crude-Oil-Cause-or-Effect-of-Inflation.html',
 'https://www.cmegroup.com/openmarkets/energy/2022/Crude-Oil-Cause-or-Effect-of-Inflation.html']

In [16]:

    
print(response)

var u = "https://www.eia.gov/todayinenergy/detail.php?id=50738#:~:text=Crude%20oil%20prices%20increased%20in%202021%20as%20increasing,global%20petroleum%20demand%20rising%20faster%20than%20petroleum%20supply.";
        if (s)
          window.location.href = u;
        else
          window.location.replace(u);
      }
      function f() {
        document.getElementById("fb").style.display = "block";
      }
      //]]>
    </script>
  </head>
  <body onload="l()">
    <div id="fb" style="display: none">
      Please <a href="https://www.bing.com/ck/a?!&&p=e8b03e386126d51fJmltdHM9MTcwMDQzODQwMCZpZ3VpZD0wNTUxOWE1OS00ZGFjLTY4YmUtMmE4My04OTk2NGNkMzY5NzImaW5zaWQ9NTQ4Mg&ptn=3&ver=2&hsh=3&fclid=05519a59-4dac-68be-2a83-89964cd36972&u=a1aHR0cHM6Ly93d3cuZWlhLmdvdi90b2RheWluZW5lcmd5L2RldGFpbC5waHA_aWQ9NTA3MzgjOn46dGV4dD1DcnVkZSUyMG9pbCUyMHByaWNlcyUyMGluY3JlYXNlZCUyMGluJTIwMjAyMSUyMGFzJTIwaW5jcmVhc2luZyxnbG9iYWwlMjBwZXRyb2xldW0lMjBkZW1hbmQlMjByaXNpbmclMjBmYXN0ZXIlMjB0aGFuJTIwcGV0cm9sZXVtJTIwc3Vw

In [29]:
res2 = AnalyseEffect(subject, 'strong economy and greater demand', model_props, 'prod', analysis_id)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
from_string grammar:



root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number_28 
number_27 ::= [-

In [48]:
ParseJSONResult(sentiment, '')

{'answer': 'Increase',
 'explanation': 'The context suggests that crude oil price is recovering, showing positive trades, and approaching key resistance levels. This indicates a potential increase in value for crude oil in the near future.',
 'numeric': 1}

In [47]:
def ParseJSONResult(response, direction):
    try:
        res = json.loads(response['result'])
        if 'answer' in res:
            answer = res['answer'].lower().replace('continue to', '').replace('remain','').strip()
            numeric = 0
            #these ones get inverted based on direction
            if answer in ['yes']:
                numeric = 1
            elif answer in ['extremely likely']:
                numeric = 0.9
            elif answer in ['very likely']:
                numeric = 0.75
            elif answer in ['likely']:
                numeric = 0.6
            elif answer in ['possible']:
                numeric = 0.45
            elif answer in ['no','unlikely']:
                answer = 0
                
            if direction.lower().strip() == 'decrease':
                numeric *= -1
    
            if answer in ['increase','grow','rise']:
                numeric = 1
            elif answer in ['decrease','decline','fall']:
                numeric = -1
            elif answer in ['fluctuate','volatile']:
                numeric = 0
            res['numeric'] = numeric
 
        return res
    except Exception as e:
        logging.exception('failed to parse json: ')
        logging.exception(response)
        logging.exception(e)
        return response

import json
with open("C:\\Data\\Code\\llm_rag\\output\\prod\\data\\2023-12-11T09-01-24.json") as f:
    data = f.read()
     
j = json.loads(data)
for subject in j.keys():
    print(subject)
    for senti in j[subject]['sentiments']:
        sentstr = j[subject]['sentiments'][senti]
        res = ParseJSONResult(sentstr, '')
        print(res)
        print()


ERROR:root:failed to parse json: 
Traceback (most recent call last):
  File "C:\Users\shave\AppData\Local\Temp\ipykernel_26256\1418869080.py", line 3, in ParseJSONResult
    res = json.loads(response['result'])
TypeError: string indices must be integers
ERROR:root:{
"answer": "Increase",
"explanation": "The context indicates that crude oil price is recovering, with positive trades and a breach of the key resistance level at 70.50. Additionally, the price needs to hold below 73.73 for the bearish trend scenario to continue, which suggests that the current negative bias may be weakening."
}
Traceback (most recent call last):
  File "C:\Users\shave\AppData\Local\Temp\ipykernel_26256\1418869080.py", line 3, in ParseJSONResult
    res = json.loads(response['result'])
TypeError: string indices must be integers
ERROR:root:string indices must be integers
Traceback (most recent call last):
  File "C:\Users\shave\AppData\Local\Temp\ipykernel_26256\1418869080.py", line 3, in ParseJSONResult
    r

Crude Oil
{
"answer": "Increase",
"explanation": "The context indicates that crude oil price is recovering, with positive trades and a breach of the key resistance level at 70.50. Additionally, the price needs to hold below 73.73 for the bearish trend scenario to continue, which suggests that the current negative bias may be weakening."
}

{
"answer": "increase",
"explanation": "The context suggests that crude oil price has been showing bullish bias, and it has support from stochastic positivity. The market is expected to recover in the second half of the year with demand exceeding supply by almost 2 mb/d. Additionally, China's rebound in oil demand growth is also a positive factor."
}

{
"answer": "continue to decline",
"explanation": "Due to ongoing concerns about global economic conditions and oil demand, as well as a shift in paper market trade to 1Q24 when markets appear to be more or less in surplus."
}

Natural Gas
{
"answer": "Decrease",
"explanation": "The current market senti